## Load required modules.

In [1]:
%matplotlib widget
# Import ncessary modules.
from sklearn.decomposition import PCA
import xlrd
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import ipywidgets 
from ipywidgets import interact, IntSlider
import ipywidgets as widgets
from matplotlib import pyplot
from IPython.display import display, clear_output, Javascript
from matplotlib.colors import LinearSegmentedColormap
from sklearn.preprocessing import FunctionTransformer

from sklearn.cluster import KMeans
from scipy.spatial import Voronoi, voronoi_plot_2d
from sklearn.pipeline import Pipeline
import gc
import psutil


import customtkinter as ctk
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog as fd
from tkinter import *

from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk
from tkinter import IntVar
from tkinter import Menu

In [2]:
# window = ctk.CTk()

# #window
# window.title('MineralAI')
# window.geometry('1100x800')

# #widgets


# window.mainloop()

In [ ]:
class main(ctk.CTk):
    def __init__(self):
        super().__init__()
        self.title("MineralAI")
        self.geometry("1600x1600")
        
        self.filtered_df = pd.DataFrame()
        self.create_widgets()
    
    def create_widgets(self):
        self.toolbox_frame = ctk.CTkFrame(self, height=20)
        self.toolbox_frame.pack(side="top", fill="x", padx=5, pady=5)

        self.selection_frame = ctk.CTkFrame(self, height=150)
        self.selection_frame.pack_propagate(False)
        self.selection_frame.pack(side="top", fill="x")

        self.output_frame = ctk.CTkFrame(self, width=300)
        self.output_frame.pack_propagate(False)
        self.output_frame.pack(side="right", fill="y", padx=5, pady=5)
        
        self.box_frame = ctk.CTkFrame(self, width = 150)
        self.box_frame.pack(side="left", fill="y", padx=5, pady=5)

        self.legend_frame = ctk.CTkFrame(self, width = 150)
        self.legend_frame.pack(side="left", fill="y", pady=5)
        
        self.bargraph_frame = ctk.CTkFrame(self)
        self.bargraph_frame.pack(side="left", fill="both", expand=True, pady=5)


        self.current_button = None

        self.select_litho_button = ctk.CTkButton(self.toolbox_frame, text="Open a File", command=self.load_data)
        self.select_litho_button.pack(side="left")

        self.lithology_label = ctk.CTkLabel(self.selection_frame, text="Select Lithologies:")
        self.lithology_label.pack(side="left", padx=5)

        self.lithology_listbox = tk.Listbox(self.selection_frame, selectmode=tk.MULTIPLE)
        self.lithology_listbox.pack(side="left", padx=5, fill="x", expand=True)

        self.apply_button = ctk.CTkButton(self.selection_frame, text="Apply", command=lambda: self.on_button_click(self.apply_button, self.filter_dataframe))
        self.apply_button.pack(side="left", padx=5)

        self.output_text = tk.Text(self.output_frame, wrap="word")
        self.output_text.pack(fill="both", expand=True)
        

        menubar = Menu(self)
        filemenu = Menu(menubar, tearoff=0)
        filemenu.add_command(label="New", command=None)
        filemenu.add_command(label="Open", command=None)
        filemenu.add_command(label="Save", command=None)
        filemenu.add_separator()
        filemenu.add_command(label="Exit", command=self.quit)
        menubar.add_cascade(label="File", menu=filemenu)
        
        helpmenu = Menu(menubar, tearoff=0)
        helpmenu.add_command(label="Help Index", command=None)
        helpmenu.add_command(label="About...", command=None)
        menubar.add_cascade(label="Help", menu=helpmenu)
        self.config(menu=menubar)



    def on_button_click(self, button, command):
        if self.current_button is not None:
            self.current_button.configure(fg_color='#3B8ED0') 

        button.configure(fg_color="grey")
        self.current_button = button

        command()


    def clear(self):
        for widget in self.winfo_children():
            if isinstance(widget, ctk.CTkEntry) or isinstance(widget, tk.Entry):
                widget.delete(0, 'end')
            elif isinstance(widget, tk.Listbox):
                widget.delete(0, tk.END)
            elif isinstance(widget, tk.Text):
                widget.delete('1.0', tk.END)

    def load_data(self):
        self.clear()
        try:
            #file_path = fd.askopenfilename()
            file_path = 'Geochemistry Results-AGG reduced variables.xlsx'
            sheet_name = 'Litogeochemistry - clean'
            self.df_0 = pd.read_excel(file_path, sheet_name=sheet_name)

            lithologies = self.df_0['Lithology'].unique()
            for lithology in lithologies:
                self.lithology_listbox.insert(tk.END, lithology)
        except:
            self.output_text.insert("end", "Invalid Selection\n")
    
    def filter_dataframe(self):
        selected_indices = self.lithology_listbox.curselection()
        selected_lithologies = [self.lithology_listbox.get(i) for i in selected_indices]
        self.filtered_df = self.df_0[self.df_0['Lithology'].isin(selected_lithologies)]
        self.remove_NaN()

    def remove_NaN(self):
        cleaned_df = self.filtered_df.iloc[:, 7:].replace('<', '', regex=True).astype(float)
        cleaned_df = cleaned_df[np.isfinite(cleaned_df).all(1)].dropna()
        self.df = cleaned_df
        self.update_output()
        self.print_pca()

    def update_output(self):
        self.output_text.delete(1.0, "end")
        self.output_text.insert("end", f"Filtered & Cleaned df:\n{self.df.head()}\n")
        self.output_text.insert("end", f"Filtered & Cleaned df:\n{self.df.shape}\n")
        
    def print_pca(self):
        try:
            self.pca_instance = PCA_class(self.df)
            self.output_text.insert("end", f"PCA performed successfully. Shape of transformed data: {self.pca_instance.x.shape}\n")
            self.output_text.insert("end", f"PCA Components:\n{self.pca_instance.pca_df.head()}\n")
            self.color_function()
            self.shape_map()
            self.loading_graph()
            
        except ValueError as e:
            self.output_text.insert("end", f"{e}\n")

    def color_function(self):
        self.dc = self.filtered_df['Lithology']
        
        self.dc = self.dc.to_frame()
        self.dc = self.dc.rename(columns={self.dc.columns[0]: 'Lithology'})
        
        # color_list = dc['Lithology'].unique()
        # default_colors = ["lime", "blue", "cyan", "deeppink", "pink", "black", "black", "black", "grey", "lightgrey"]
        # color_map = {}
        # # Populate the dictionary with symbols for each shape
        # for i, color in enumerate(color_list):
        #     if i < len(default_colors):
        #         color_map[color] = default_colors[i]
        
        self.color_map = {
            'Calcitic marble': 'lime',
            'Dolomitic marble': 'blue',
            'Di-Tr dolomitic marble': 'cyan',
            'Carbonatite': 'deeppink',
            'Carbonatite-like': 'pink',
            'Syenite': 'black',
            'Altered Syenite': 'black',
            'Syenite-like': 'black',
            'Impure Siliciclastic': 'grey',
            'Pure Siliciclastic': 'lightgrey'
        }
        
        self.dc['Color'] = self.dc['Lithology'].map(self.color_map)

        self.output_text.insert("end", self.dc)

    def shape_map(self):
        self.ds = self.filtered_df.iloc[:, 4]
        self.ds = self.ds.to_frame()
        self.ds = self.ds.rename(columns={self.ds.columns[0]: 'Shape'})
        
        # shape_list = ds['Shape'].unique()
        # default_symbols = ["^", "*", "o", "s", "D"]
        # color_map1 = {}
        # # Populate the dictionary with symbols for each shape
        # for i, shape in enumerate(shape_list):
        #     if i < len(default_symbols):
        #         color_map1[shape] = default_symbols[i]
        
        self.color_map1 = {
            'Marble Units': "^",
            'Altered Intrusion': "*",
            'Siliciclastic': "o",
            'Intrusion': "s",
            'Anamolous Rock': "D",
        }
        
        self.ds["Shapes"] = self.ds["Shape"].map(self.color_map1)
        self.output_text.insert("end", self.ds)

            
    def loading_graph(self):
        loading_class(self.pca_instance, self.df, self.bargraph_frame, self.box_frame, self.on_button_click, self.apply_button)
        threed_class(self.pca_instance, self.df, self.bargraph_frame, self.box_frame, self.dc, self.ds, self.color_map, self.color_map1, self.legend_frame, self.on_button_click, self.apply_button)




class PCA_class:
    def __init__(self, df):
        self.df = df
        self.perform_pca()
        self.bargraph_frame = None
        
    def perform_pca(self):
        #self.scaling=StandardScaler()
        scaling = FunctionTransformer(np.log10, validate=True)
        try:
            scaling.fit(self.df)
            Scaled_data = scaling.transform(self.df)
            Scaled_df = pd.DataFrame(Scaled_data)
            
            self.pca = PCA(n_components=6)
            self.pca.fit(Scaled_data)
            self.x = self.pca.transform(Scaled_data)
            self.create_pca_df()
             
        except Exception as e:
            raise ValueError('Not enough components for PCA. Please select more!') from e

    def create_pca_df(self):  
        self.pca_df = pd.DataFrame(
            data=self.x, 
            columns=['PC'+str(i) for i in range(1, len(self.pca.components_)+1)]
        )
        self.scale_pca()
        
    def scale_pca(self):
        self.pca_df_scaled = self.pca_df.copy()
        scaler_df = self.pca_df[self.pca_df.columns]
        scaler = 1 / (scaler_df.max() - scaler_df.min())
        for index in scaler.index:
            self.pca_df_scaled[index] *= scaler[index]
            


class loading_class:
    def __init__(self, pca_instance, df, bargraph_frame, box_frame, on_button_click, apply_button):
        self.apply_button = apply_button
        self.on_button_click = on_button_click
        self.pca = pca_instance.pca
        self.df = df
        self.loadings = pd.DataFrame(self.pca.components_.T, columns=['PC'+str(i) for i in range(1, len(self.pca.components_)+1)], index=df.columns)
        self.bargraph_frame = bargraph_frame
        self.box_frame = box_frame
        self.var = IntVar()
        #self.var.trace_add("write", self.check)
        self.check()
        self.create_checkbox()

    
        
    def check(self):
        self.graph_bar_button = ctk.CTkButton(self.box_frame, text="Graph PC bar graph", command=lambda: self.on_button_click(self.graph_bar_button, self.update_plots))
        self.graph_bar_button.pack(side="top", padx=5, pady=5)

    def create_checkbox(self):
        self.checkbox = ctk.CTkCheckBox(self.box_frame, text="Sort Loadings", variable=self.var)
        self.checkbox.pack(side="top", anchor="w", padx=5, pady=5)


    def normalize(self, values, vmin=None, vmax=None):
        vmin = vmin if vmin is not None else np.min(values)
        vmax = vmax if vmax is not None else np.max(values)
        norm_values = (values - vmin) / (vmax - vmin)
        return norm_values

    def plot_pc_barchart(self, data, pc_name, ax, sort):
        if sort:
            data = data.sort_values(ascending=False)
        norm_values = self.normalize(data, vmin=-max(abs(data)), vmax=max(abs(data)))
        bar_colors = plt.cm.coolwarm(norm_values)
        data.plot(kind='barh', color=bar_colors, ax=ax, fontsize="x-small")
        ax.set_title(pc_name)
        ax.grid(True, linestyle='--', linewidth=0.5)
        ax.set_xlabel('')
        ax.set_ylabel('Elements')

    def update_plots(self):
        sort = self.var.get() == 1
        plt.close('all')
        for widget in self.bargraph_frame.winfo_children():
            widget.destroy() 


        fig, axes = plt.subplots(ncols=len(self.loadings.columns), figsize=(12, 8))

        for i in range(len(self.loadings.columns)):
            self.plot_pc_barchart(self.loadings[f'PC{i+1}'], f'PC{i+1}', axes[i], sort)

        plt.tight_layout()

        fig_c = plt.figure(figsize=(12, 1))
        min_val = self.loadings.min().min()
        max_val = self.loadings.max().max()
        norm = plt.Normalize(vmin=min_val, vmax=max_val)
        cmap = plt.cm.coolwarm
        cbar = fig_c.colorbar(plt.cm.ScalarMappable(norm=norm, cmap=cmap), ax=axes, orientation='horizontal', fraction=0.02)

        canvas = FigureCanvasTkAgg(fig, master=self.bargraph_frame)
        canvas.draw()
        canvas.get_tk_widget().pack(side=ctk.TOP, fill=ctk.BOTH, expand=True)




class threed_class:    
    def __init__(self, pca_instance, df, bargraph_frame, box_frame, dc, ds, color_map, color_map1, legend_frame, on_button_click, apply_button):
        self.apply_button = apply_button
        self.on_button_click = on_button_click
        self.pca = pca_instance.pca
        self.pca_df_scaled = pca_instance.pca_df_scaled
        self.df = df
        self.dc = dc
        self.ds = ds
        self.color_map = color_map
        self.color_map1 = color_map1

        self.loadings = pd.DataFrame(self.pca.components_.T, columns=['PC'+str(i) for i in range(1, len(self.pca.components_)+1)], index=df.columns)
        self.bargraph_frame = bargraph_frame
        self.box_frame = box_frame
        self.legend_frame = legend_frame
        self.var = IntVar()
        self.var1 = IntVar()
        # Plot 3D scatterplot of PCA
        #df = pd.read_excel(file_path, sheet_name=sheet_name)
        #description = df.iloc[:, 0]
        #descriptions = []
        dc.reset_index(drop=True, inplace=True)
        ds.reset_index(drop=True, inplace=True)
        self.plot_3d()
        self.checkbox()

    def plot_3d(self):
        self.plot_3d_button = ctk.CTkButton(self.box_frame, text="Graph 3d", command=lambda: self.on_button_click(self.plot_3d_button, self.show_shape))
        self.plot_3d_button.pack(side="top", padx=5, pady=5)
        
    def checkbox(self):
        self.checkbox = ctk.CTkCheckBox(self.box_frame, text="Display shape", variable=self.var)
        self.checkbox.pack(side="top", anchor="w", padx=5)
        
        self.checkbox1 = ctk.CTkCheckBox(self.box_frame, text="Display Legend", variable=self.var1)
        self.checkbox1.pack(side="top", anchor="w", padx=5)

        self.multiselect_3d = tk.Listbox(self.box_frame, selectmode=tk.MULTIPLE)
        self.multiselect_3d.pack(side="top", padx=5, pady=5)
        for names in self.df.columns.tolist():
                self.multiselect_3d.insert(tk.END, names)
                

        
    def show_shape(self): 
        plt.close('all')
        for widget in self.bargraph_frame.winfo_children():
            widget.destroy() 

    
            
        fig = plt.figure(figsize=(6, 7))
        self.ax = fig.add_subplot(111, projection='3d')
        xdata = self.pca_df_scaled['PC1']
        ydata = self.pca_df_scaled['PC2']
        zdata = self.pca_df_scaled['PC3'] 

        canvas = FigureCanvasTkAgg(fig, master=self.bargraph_frame)
        canvas.draw()
        
        toolbar = NavigationToolbar2Tk(canvas, self.bargraph_frame)
        toolbar.update()
        toolbar.pack(side=tk.TOP, fill=tk.X)
        canvas.get_tk_widget().pack(side=ctk.RIGHT, fill=ctk.Y)
        
        self.shapes1 = []
        for i in range(len(self.dc)):
            shape1 = self.ax.scatter3D(xdata[i], ydata[i], zdata[i], c=self.dc['Color'][i], marker=self.ds["Shapes"][i])
            self.shapes1.append(shape1)
            #text = ax.text(xdata[i], ydata[i], zdata[i], description[i], size=10, zorder=1, color='k')
            #descriptions.append(text)
        
        self.dots = self.ax.scatter3D(xdata, ydata, zdata, c=self.dc['Color'])

        self.setups()
        self.shape()
        self.legend()

        
    def shape(self):
        shape = self.var.get() == 1
        if shape:
            for shape1 in self.shapes1:
                shape1.set_visible(True)  
            self.dots.set_visible(False) 
        else:
            self.dots.set_visible(True)
            for shape1 in self.shapes1:
                shape1.set_visible(False)
        
    def legend(self):
        figx = plt.figure(figsize=(2, 8))
        axx = figx.add_subplot(111)
        axx.axis('off')
        
        handle1 = []
        label1 = []
        for lithology, color in self.color_map.items():
            l1 = self.ax.scatter([], [], c=color, label=lithology)
            handle1.append(l1)
            label1.append(lithology)
        
        handle2 = []
        label2 = []
        for shape, shapes in self.color_map1.items():
            l2 = self.ax.scatter([], [], c="black", marker=shapes, label=shape)
            handle2.append(l2)
            label2.append(shape)
        
        self.legend1 = pyplot.legend(handle1, label1, loc = "upper left", title='Lithology', fontsize='x-small')
        self.legend2 = pyplot.legend(handle2, label2, loc = "lower left", title='Shapes', fontsize='x-small')
        axx.add_artist(self.legend1)
        axx.add_artist(self.legend2)
        self.canvas1 = FigureCanvasTkAgg(figx, master=self.bargraph_frame)
        self.canvas1.get_tk_widget().pack(side="left", fill="both", expand=True)
        
        self.legend1.set_visible(False)
        self.legend2.set_visible(False)
        self.legenda()
        
    def legenda(self):
        Legend = self.var1.get() == 1
        if Legend:
            self.legend1.set_visible(True)  
            self.legend2.set_visible(True)
        else:
            self.legend1.set_visible(False)
            self.legend2.set_visible(False)

    

    def setups(self):
        
        #set up loadings
        self.show_names=self.df.columns.tolist()
        self.indx=self.df.columns.get_indexer(self.show_names)
        scale=2
        
        # Define the x, y, z variables
        loadings_1 = self.pca.components_
        self.xs = scale*loadings_1[0]
        self.ys = scale*loadings_1[1]
        self.zs = scale*loadings_1[2]  
        # Plot title of graph
        plt.title(f'3D Biplot')
          
        # Plot x, y, z labels
        self.ax.set_xlabel('PC1')
        self.ax.set_ylabel('PC2')
        self.ax.set_zlabel('PC3')
        x_arr = np.zeros(len(loadings_1[0]))
        y_arr = z_arr = x_arr
        
        all_points = np.concatenate((self.xs, self.ys, self.zs))
        min_value = np.min(all_points)
        max_value = np.max(all_points)
        axis_range = [min_value, max_value]
        
        self.ax.plot([x_arr.any() + axis_range[0], x_arr.any() + axis_range[1]], [y_arr.any(), y_arr.any()], [z_arr.any(), z_arr.any()], color='k', linestyle='--')
        self.ax.plot([x_arr.any(), x_arr.any()], [y_arr.any() + axis_range[0], y_arr.any() + axis_range[1]], [z_arr.any(), z_arr.any()], color='k', linestyle='--') 
        self.ax.plot([x_arr.any(), x_arr.any()], [y_arr.any(), y_arr.any()], [z_arr.any() + axis_range[0], z_arr.any() + axis_range[1]], color='k', linestyle='--')
        
        self.arrow_list = []
        
        #show_arrows = ax.quiver(x_arr[indx], y_arr[indx], z_arr[indx], xs[indx], ys[indx], zs[indx],color='r')
        for i, name in enumerate(self.df.columns):
            ip = self.df.columns.get_loc(name)
            arrow = self.ax.quiver(x_arr[ip], y_arr[ip], z_arr[ip], self.xs[ip], self.ys[ip], self.zs[ip], color='r', arrow_length_ratio = 0.1, linewidth=0.5)
            self.arrow_list.append(arrow)
            
        # Plot the loadings
        self.namelist = []
        for i , names in enumerate(self.show_names):
            ip=self.indx[i]
            #ax.scatter(xs[ip], ys[ip], zs[ip], s=100)
            n = self.ax.text(self.xs[ip]+0.02, self.ys[ip]+0.02, self.zs[ip]+0.02, names, fontsize='small')
            self.namelist.append(n)

        selected_indices = self.multiselect_3d.curselection()
        self.selected_items = [self.multiselect_3d.get(i) for i in selected_indices]
        self.findname_specific()
     
    def findname_specific(self):
        options = self.df.columns.tolist()
        for i in range(len(options[:])):  
            if options[i] in self.selected_items:
                self.namelist[i].set_visible(True)
                self.arrow_list[i].set_visible(True)
            else:
                self.namelist[i].set_visible(False)
                self.arrow_list[i].set_visible(False)
    


    
#     display(select_multiple)
    
if __name__ == "__main__":
    app = main()
    app.mainloop()






## Display loadings.

## Plot a 3D scatter plot with arrows for PCA directions.

In [4]:
memory_1 = get_memory_usage()

# Initialize the 3D graph
plt.close('all')
fig = plt.figure(figsize=(9, 9))
ax = fig.add_subplot(111, projection='3d')
  
# Define scaled features as arrays
xdata = pca_df_scaled['PC1']
ydata = pca_df_scaled['PC2']
zdata = pca_df_scaled['PC3']

# Plot 3D scatterplot of PCA

#df = pd.read_excel(file_path, sheet_name=sheet_name)
#description = df.iloc[:, 0]

#descriptions = []
dc.reset_index(drop=True, inplace=True)
ds.reset_index(drop=True, inplace=True)


#show shape
shapes1 = []
for i in range(len(filtered_df)):
    #print(f"Index {i}: Color {dc['Color'][i]}, Shape {ds['Shapes'][i]}")  # Debug print
    shape1 = ax.scatter3D(xdata[i], ydata[i], zdata[i], c=dc['Color'][i], marker=ds["Shapes"][i])
    shapes1.append(shape1)
    #text = ax.text(xdata[i], ydata[i], zdata[i], description[i], size=10, zorder=1, color='k')
    #descriptions.append(text)

dots = ax.scatter3D(xdata, ydata, zdata, c=dc['Color'])
dots.set_visible(True)

def shape(Show_Shape):
    if Show_Shape:
        for shape1 in shapes1:
            shape1.set_visible(True)  
        dots.set_visible(False) 
    else:
        dots.set_visible(True)
        for shape1 in shapes1:
            shape1.set_visible(False)

interact(shape, Show_Shape=False)


#create legend
handle1 = []
label1 = []
for lithology, color in color_map.items():
    l1 = ax.scatter([], [], c=color, label=lithology)
    handle1.append(l1)
    label1.append(lithology)

handle2 = []
label2 = []
for shape, shapes in color_map1.items():
    l2 = ax.scatter([], [], c="black", marker=shapes, label=shape)
    handle2.append(l2)
    label2.append(shape)

legend1 = pyplot.legend(handle1, label1, bbox_to_anchor=(0.03, 0.2), title='Lithology', fontsize='x-small')
legend2 = pyplot.legend(handle2, label2, bbox_to_anchor=(0.03, 0.2), title='Shapes', fontsize='x-small')
ax.add_artist(legend1)
ax.add_artist(legend2)

def legenda(Legend):
    if Legend:
        legend1.set_visible(True)  
        legend2.set_visible(False)
    else:
        legend1.set_visible(False)
        legend2.set_visible(True)
interact(legenda, Legend=[('Shapes Legend', True),('Lithology Legend', False)])


#set up loadings
show_names=df.columns.tolist()
indx=df.columns.get_indexer(show_names)
scale=2

# Define the x, y, z variables
loadings_1 = pca.components_
xs = scale*loadings_1[0]
ys = scale*loadings_1[1]
zs = scale*loadings_1[2]  
# Plot title of graph
plt.title(f'3D Biplot')
  
# Plot x, y, z labels
ax.set_xlabel('PC1')#, rotation=150)
ax.set_ylabel('PC2')
ax.set_zlabel('PC3')#, rotation=60)
#plt.savefig('3D_biplot.svg')



# Plot the arrows
arrow_list = []
x_arr = np.zeros(len(loadings_1[0]))
y_arr = z_arr = x_arr

#show_arrows = ax.quiver(x_arr[indx], y_arr[indx], z_arr[indx], xs[indx], ys[indx], zs[indx],color='r')
for i, name in enumerate(df.columns):
    ip = df.columns.get_loc(name)
    arrow = ax.quiver(x_arr[ip], y_arr[ip], z_arr[ip], xs[ip], ys[ip], zs[ip], color='r', arrow_length_ratio = 0.1, linewidth=0.5)
    arrow_list.append(arrow)
    
def findarrow(change):
    if change: 
        for a in arrow_list:
            a.set_visible(True)
    else:
        for a in arrow_list:
            a.set_visible(False)


#plot the axis
all_points = np.concatenate((xs, ys, zs))
min_value = np.min(all_points)
max_value = np.max(all_points)

axis_range = [min_value, max_value]

ax.plot([x_arr.any() + axis_range[0], x_arr.any() + axis_range[1]], [y_arr.any(), y_arr.any()], [z_arr.any(), z_arr.any()], color='k', linestyle='--')
ax.plot([x_arr.any(), x_arr.any()], [y_arr.any() + axis_range[0], y_arr.any() + axis_range[1]], [z_arr.any(), z_arr.any()], color='k', linestyle='--') 
ax.plot([x_arr.any(), x_arr.any()], [y_arr.any(), y_arr.any()], [z_arr.any() + axis_range[0], z_arr.any() + axis_range[1]], color='k', linestyle='--')




# Plot the loadings
namelist = []

for i , names in enumerate(show_names):
    ip=indx[i]
    #ax.scatter(xs[ip], ys[ip], zs[ip], s=100)
    n = ax.text(xs[ip]+0.02, ys[ip]+0.02, zs[ip]+0.02, names, fontsize='small')
    namelist.append(n)

show_name_checkbox = widgets.Checkbox(value=False, description='Show all Names')
all_arrow_checkbox = widgets.Checkbox(value=False, description='Show all Arrows')

def findname(show_name):
    if show_name:
        for n in namelist:
            n.set_visible(True)  
    else:
        for n in namelist:
            n.set_visible(False)


#create multiselection list for individual elements
def findname_specific(change):
    selected_items = change['new']
    # if 'all' in selected_items:
    #     select_multiple.value = options
    #     for n in namelist:
    #         n.set_visible(True)
    #     for arrow in arrow_list:
    #         arrow.set_visible(True)
    # else:
    for i in range(len(options[1:])):  
        if options[i] in selected_items:
            namelist[i].set_visible(True)
            arrow_list[i].set_visible(True)
        else:
            namelist[i].set_visible(False)
            arrow_list[i].set_visible(False)
    show_name_checkbox.value = False
    all_arrow_checkbox.value = False
plt.draw()

interact(findname, show_name=show_name_checkbox)
interact(findarrow, change=all_arrow_checkbox)

options = df.columns.tolist()

select_multiple = widgets.SelectMultiple(
    options=options,
    value=[],
    description='Select multiple items:'
)
select_multiple.observe(findname_specific, 'value')
print ('Press Command to Select Multiple')

display(select_multiple)


#memory
memory_2 = get_memory_usage()
gc.collect()
memory_3 = get_memory_usage()

print(f"Memory Usage initial 3d: {memory_1:.2f} MB")
print(f"Memory Usage After 3d Plot: {memory_2:.2f} MB")
print(f"Memory Usage After Deletion: {memory_3:.2f} MB")


NameError: name 'get_memory_usage' is not defined

In [ ]:
#prediction
# from sklearn.svm import SVC
# from sklearn.pipeline import make_pipeline
# try:
#     y = dc['Lithology']
#     y = y.values.ravel()
#     X = pca_df_scaled
#     print("Shape of y:", y.shape)
#     print("Shape of X:", X.shape)
    
#     clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
#     clf.fit(X, y)
    
#     print(clf.predict([[0.3, -0.01, 0.1]]))
# except:
#     print ("need more litho group")

# memory_4 = get_memory_usage()
# print(f"Memory Usage After predict: {memory_4:.2f} MB")

In [ ]:
fig = plt.figure(figsize=(10, 9))

# create 2d plot for each PC set
def planes (options):
    #create graph and legend
    plt.clf()
    ax1 = fig.add_subplot(111)
    handle3 = []
    label3 = []
    for lithology, color in color_map.items():
        l3 = ax1.scatter([], [], c=color, label=lithology)
        handle3.append(l3)
        label3.append(lithology)
    
    handle4 = []
    label4 = []
    for shape, shapes in color_map1.items():
        l4 = ax1.scatter([], [], c="black", marker=shapes, label=shape)
        handle4.append(l4)
        label4.append(shape)
        
    legend3 = ax1.legend(handle3, label3, title='Lithology', fontsize='x-small')
    legend4 = ax1.legend(handle4, label4, loc = "lower left", title='Shapes', fontsize='x-small')
    ax1.add_artist(legend3)
    ax1.add_artist(legend4)
    
    
    #show shape checkbox
    xdata = pca_df_scaled[options[0]]
    ydata = pca_df_scaled[options[1]]
    
    shapes2 = []
    for i in range(len(df)):
        shape2 = plt.scatter(xdata[i], ydata[i], c=dc['Color'][i], marker=ds["Shapes"][i])
        shapes2.append(shape2)
    
    dots2 = plt.scatter(xdata, ydata, c=dc['Color'])
    dots2.set_visible(True)
    
    def shape(Show_Shape):
        if Show_Shape:
            for shape2 in shapes2:
                shape2.set_visible(True)  
            dots2.set_visible(False) 
        else:
            dots2.set_visible(True)
            for shape2 in shapes2:
                shape2.set_visible(False)
    
    interact(shape, Show_Shape=False)
    


    #set up names and location
    show_names=df.columns.tolist()
    indx=df.columns.get_indexer(show_names)
    
    
    loadings = pca.components_
    xs = loadings[options[2]]
    ys = loadings[options[3]]
    
    
    #Plot the arrows
    arrow_list_2d = []
    
    x_arr = np.zeros(len(loadings[0]))
    y_arr = x_arr

    plt.axhline(y=y_arr.any(), color='k', linestyle='--')
    plt.axvline(x=x_arr.any(), color='k', linestyle='--')
    
    arrow_scale=1.2
    
    for i, name in enumerate(df.columns):
        ip = df.columns.get_loc(name)
        arrow_2d = plt.quiver(x_arr[ip], y_arr[ip], xs[ip], ys[ip], color='r', scale = arrow_scale, width = 0.002)
        arrow_list_2d.append(arrow_2d)
        
    namelist_2d = []
    for i , names in enumerate(show_names):
        ip=indx[i]
        #ax.scatter(xs[ip], ys[ip], zs[ip], s=100)
        n = plt.text((1)*xs[ip], (1)*ys[ip], names, fontsize='small')
        #n = plt.text(xs[ip], ys[ip], names, fontsize='small')
        namelist_2d.append(n)

    # def findname_2d(show_name):
    #     if show_name:
    #         for n in namelist_2d:
    #             n.set_visible(True)  
    #     else:
    #         for n in namelist_2d:
    #             n.set_visible(False)
        
    # def findarrow_2d(Visibility_arrow):
    #     if Visibility_arrow: 
    #         for a in arrow_list_2d:
    #             a.set_visible(True)
    #     else:
    #         for a in arrow_list_2d:
    #             a.set_visible(False)

    
    # show_name_checkbox = widgets.Checkbox(value=False, description='Show all Names')
    # all_arrow_checkbox = widgets.Checkbox(value=False, description='Show all Arrows')
    
    #create multiselection list for individual elements
    def findname_specific_2d(change):
        selected_items = change['new']
        for i in range(len(options_ele[:])): 
            if options_ele[i] in selected_items:
                namelist_2d[i].set_visible(True)
                arrow_list_2d[i].set_visible(True)
            else:
                namelist_2d[i].set_visible(False)
                arrow_list_2d[i].set_visible(False)
        show_name_checkbox.value = False
        all_arrow_checkbox.value = False
    plt.draw()
    
    # interact(findname_2d, show_name=show_name_checkbox)
    # interact(findarrow_2d, Visibility_arrow=all_arrow_checkbox)
    
    options_ele = df.columns.tolist()
    
    select_multiple = widgets.SelectMultiple(
        options=options_ele,
        value=[],
        description='Select multiple items:'
    )
    select_multiple.observe(findname_specific_2d, 'value')
    print ('Press Command to Select Multiple')
    
    display(select_multiple)
    plt.xlabel(options[0])
    plt.ylabel(options[1])
    plt.title(f"{options[0]} vs {options[1]}")


#create a combination of PC sets
list_plane = []
for j in range (len(pca_df.columns)):
    for i in range (j+1, len(pca_df.columns)):
        list_plane.append((f'PC{j+1} vs PC{i+1}', [f'PC{j+1}', f'PC{i+1}', j, i]))
                    
widgets.interact(planes, options = list_plane)
memory_5 = get_memory_usage()

print(f"Memory Usage After 2d PC: {memory_5:.2f} MB")

In [ ]:
# # import relevant libraries for 3d graph
# from mpl_toolkits.mplot3d import Axes3D
# fig = plt.figure(figsize=(10,10))
 
# # choose projection 3d for creating a 3d graph
# axis = fig.add_subplot(111, projection='3d')
 
# # x[:,0]is pc1,x[:,1] is pc2 while x[:,2] is pc3
# img=axis.scatter(x[:,0],x[:,1],x[:,2], c=df['Zn_ppm'])

# # Plot the loadings
# for i in range(5):
#     axis.scatter(xs[i], ys[i], zs[i], s=200)
#     #axis.text(
#     #    xs[i] + 0.1, 
#     #    ys[i] + 0.1, 
#     #    zs[i] + 0.1, 
#     #    varnames)
  
# # Plot the arrows
# x_arr = np.zeros(len(loadings[0]))
# y_arr = z_arr = x_arr
# axis.quiver(x_arr, y_arr, z_arr, xs, ys, zs)

# axis.set_xlabel("PC1")
# axis.set_ylabel("PC2")
# axis.set_zlabel("PC3")
# fig.colorbar(img)

In [ ]:
# Remove all rows that have at least one NaN.
df = df.dropna()

max_Zn = df['Zn_ppm'].max()
min_Zn = df['Zn_ppm'].min()

def cluster_2d(axis1, axis2):
    global df
    if axis1 != axis2:
        # Remove "<" from the values.
        df[axis1] = df[axis1].apply(lambda x: x.replace('<', '') if isinstance(x, str) and '<' in x else x)
        df[axis2] = df[axis2].apply(lambda x: x.replace('<', '') if isinstance(x, str) and '<' in x else x)
        
        # Change to "object" to "float".
        df[axis1] = df[axis1].astype(float)
        df[axis2] = df[axis2].astype(float)
        
        
        # Set the data for clustering
        X = df[[axis1, axis2]].values
        
        #scaler = StandardScaler()
        #X_scaled = scaler.fit_transform(X)
        dd = pd.read_excel(file_path, sheet_name=sheet_name)
        
        #scaler and model
        def update_kmeans(k):    
            pipe = Pipeline([
            ("scale", StandardScaler()),
            ("model", KMeans(n_clusters=k, random_state = 0, n_init='auto'))])
        
            pipe.fit(X)
        
            # Add cluster labels to the DataFrame
            df['cluster'] = pipe.named_steps['model'].labels_

            #create meshgrid for background coloring
            x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
            y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
            xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.01),
                                 np.arange(y_min, y_max, 0.01))
            
            #predict cluster for each point in the meshgrid
            Z = pipe.named_steps['model'].predict(pipe.named_steps['scale'].transform(np.c_[xx.ravel(), yy.ravel()]))
            Z = Z.reshape(xx.shape)

            #draw
            plt.figure(figsize=(9, 7))
            plt.contourf(xx, yy, Z, cmap=plt.cm.Pastel1, alpha=0.2)

            #create a size chart corrisponding to the value of Zn_ppm and plot
            def map_size(x, new_min, new_max):
                old_min = min_Zn
                old_max = max_Zn
                return ((np.log(x + 1) - np.log(old_min + 1)) / (np.log(old_max + 1) - np.log(old_min + 1))) * (new_max - new_min) + new_min
            
            sns.scatterplot(data=df, x=axis1, y=axis2, c=dc['Color'], s=df['Zn_ppm'].apply(lambda x: map_size(x, 20, 100)))# change the last two values if want to change size range of point
        
            #plot the sample names
            df.apply(lambda row: plt.text(row[axis1], row[axis2] + 0.1, dd.iloc[row.name, 0], fontsize=7, ha='center'), axis=1)
        
            #plot the controids
            scaler = pipe.named_steps['scale']
            centroids = pipe.named_steps['model'].cluster_centers_
            centroids_original = scaler.inverse_transform(centroids)
        
            plt.scatter(centroids_original[:,0], centroids_original[:,1], 
                    marker="X", c="red", s=30, label="centroids")
            
            plt.legend()
            plt.title("K-Means Clustering")
            plt.show()
            
        #create interact slider for k
        interact(update_kmeans, k = IntSlider(min=2, max=10, step=1, description='N Clusters'))
    else:
        #debugging if two selection the same
        print ('select two different elements')

#creating the dropdown of elements
interact(cluster_2d, 
         axis1=widgets.Dropdown(options=df.columns.tolist(), value=df.columns[32], description='Axis1'), 
         axis2=widgets.Dropdown(options=df.columns.tolist(), value=df.columns[34], description='Axis2'))

memory_6 = get_memory_usage()
print(f"Memory Usage After 2d cluster: {memory_6:.2f} MB")


In [ ]:
# Remove all rows that have at least one NaN.
df = df.dropna()
dd = pd.read_excel(file_path, sheet_name=sheet_name)

def cluster_3d(axis1, axis2, axis3):
    global df
    #make sure three elements are different
    if axis1 != axis2 and axis1 != axis3 and axis2 != axis3:
        #cleaning
        for col in [axis1, axis2, axis3]:
            df[col] = df[col].apply(lambda x: x.replace('<', '') if isinstance(x, str) and '<' in x else x)
        df = df.astype({axis1: 'float', axis2: 'float', axis3: 'float'})

        # Set the data for clustering
        X = df[[axis1, axis2, axis3]].values
        
        #scaler = StandardScaler()
        #X_scaled = scaler.fit_transform(X)
        
        #scaler and model
        def update_kmeans(k=3):
            pipe = Pipeline([
                ("scale", StandardScaler()),
                ("model", KMeans(n_clusters=k, random_state=0, n_init='auto'))
            ])
            pipe.fit(X)

            # Add cluster labels to the DataFrame
            df['cluster'] = pipe.named_steps['model'].labels_
        
            fig = plt.figure(figsize=(10, 8))
            ax = fig.add_subplot(111, projection='3d')
            
            #plot the data points
            scatter = ax.scatter(df[axis1], df[axis2], df[axis3], c=df['cluster'], cmap='viridis', s=10)
            df.apply(lambda row: ax.text(row[axis1], row[axis2], row[axis3], dd.iloc[row.name, 0], fontsize=7, ha='center'), axis=1)
        
        
            # Plot the centroids
            scaler = pipe.named_steps['scale']
            centroids = pipe.named_steps['model'].cluster_centers_
            centroids_original = scaler.inverse_transform(centroids)
            
            ax.scatter(centroids_original[:, 0], centroids_original[:, 1], centroids_original[:, 2], 
                       marker="X", c="red", s=100, label="centroids")
        
            ax.set_xlabel(axis1)
            ax.set_ylabel(axis2)
            ax.set_zlabel(axis3)
            plt.legend()
            plt.title("3D K-Means Clustering")
            plt.show()
        
        interact(update_kmeans, k=IntSlider(min=2, max=10, step=1, description='N Clusters'))
    else:
        #debugging if two or more elements the same
        print ('select three different elements')

#create the dropdown for the elements
interact(cluster_3d, 
         axis1=widgets.Dropdown(options=df.columns.tolist(), value=df.columns[0], description='Axis 1'), 
         axis2=widgets.Dropdown(options=df.columns.tolist(), value=df.columns[2], description='Axis 2'), 
         axis3=widgets.Dropdown(options=df.columns.tolist(), value=df.columns[4], description='Axis 3'))


memory_7 = get_memory_usage()
print(f"Memory Usage After 3d cluster: {memory_7:.2f} MB")


In [ ]:
            #3d background shading, use only if enough memory (mac impossible)
            # x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
            # y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
            # z_min, z_max = X[:, 2].min() - 1, X[:, 2].max() + 1
            # xx, yy, zz = np.meshgrid(np.linspace(x_min, x_max, 50),
            #                          np.linspace(y_min, y_max, 50),
            #                          np.linspace(z_min, z_max, 50))
        
            # # Predict cluster labels for each point in the mesh grid
            # grid_points = np.c_[xx.ravel(), yy.ravel(), zz.ravel()]
            # grid_points_scaled = pipe.named_steps['scale'].transform(grid_points)
            # Z = pipe.named_steps['model'].predict(grid_points_scaled)
            # Z = Z.reshape(xx.shape)
        
            # ax.scatter(xx, yy, zz, c=Z, cmap='Pastel1', alpha=0.1)

In [ ]:
# Plot the distortion score elbow.
from yellowbrick.cluster import KElbowVisualizer

plt.figure(figsize=(10, 6))
model = KMeans(n_init='auto')
visualizer = KElbowVisualizer(model, k=(1, 12)).fit(X)
visualizer.show()